In [245]:
import comet_ml
experiment = comet_ml.Experiment(api_key = "1OR6MquthzWmNUZ37ADRp4uWG", project_name = "a-photo-a-day")

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/jacobsela/a-photo-a-day/ea2f4a45ddc34d43b69de4b651413e9f
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     batch_loss [161]               : (0.018330425024032593, 0.050208356231451035)
COMET INFO:     batch_mean_squared_error [161] : (0.019251085817813873, 0.050208356231451035)
COMET INFO:     epoch_duration [9]             : (0.18015733300035208, 5.83610296400002)
COMET INFO:     loss [9]                       : (0.018540554866194724, 0.02277167149254533)
COMET INFO:     mean_squared_error [9]         : (0.022263336926698685, 0.023175694048404694)
COMET INFO:   Others [count]:
COMET INFO:     trainable_params [3] : 2774316
COMET INFO:   Parameters:
COMET INFO:     Adam_amsgrad       : 1
COMET INFO:     Adam_beta_1        : 0.9
COMET INFO:

In [246]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import sklearn
import os
from pathlib import Path

In [247]:
import random

random.seed(2)

In [248]:
print(tf.__version__)

2.0.0


In [249]:
# Load Data

# data is a list of lists where the first element is the age and the second is the latent representation

# i.e. data[0] contains the entire first video
# data[0][0] is a list of two values (the first age of the video, and the latent representation of the photo at that age)

data = []

video = []

paths = ["video1_latent_representations",
         "video2_latent_representations",
         "video3_latent_representations"]

for path in paths:
    
    for photo in os.listdir(path):

        latent_representation = np.load(path + "/" + photo).flatten()
        age = int(Path(photo).stem)

        video.append([age, latent_representation])
        video.sort()
        
    data.append(video)
    video = []

In [250]:
differences = []

for person in data:
    for j, photo in enumerate(person):
        if j == 0:
            continue
        differences.append(photo[1] - person[j - 1][1])

differences = np.array(differences)

differences_aux = differences.transpose()

variances = []

for values in differences_aux:
    variances.append(np.var(values))
    
# list of components sorted by variance
smallest_variance_components = np.argsort(variances)

np.save("svc", smallest_variance_components)

In [251]:
model = keras.Sequential([
    keras.layers.Input(512*18 + 2),
    keras.layers.Dense(100, activation='tanh', kernel_regularizer = keras.regularizers.l1(0.0000)),
    keras.layers.Dense(512*18)
])

In [252]:
model.compile(optimizer='adam',
              loss=['mean_squared_error'],
              metrics=['mean_squared_error'])

In [253]:
#X = np.concatenate((np.array([data[2][0][0],data[2][len(data[2]) - 1][0]]), data[2][0][1]))
#Y = data[2][len(data[2]) - 1][1]
#print(X.shape)
#print(Y.shape)

In [254]:
iterations = 10
total_num_photos = iterations * (len(data[0]) + len(data[1]) + len(data[2]))
X = np.empty(shape=(total_num_photos, 512*18+2))
Y = np.empty(shape=(total_num_photos, 512*18))

i = 0

for iteration in range(iterations):
    for person in data:
        for j, photo in enumerate(person):
            while (abs(y_aux - j) < 10):
                y_aux = random.randint(0, len(person) - 1)
            Y[i] = person[y_aux][1]
            X[i] = np.concatenate((np.array([photo[0],person[y_aux][0]]), photo[1]))
            i += 1


In [255]:
model.fit(np.array(X), np.array(Y), epochs=3)

Train on 6100 samples
Epoch 1/3
6100/6100 [==============================] - 4s 648us/sample - loss: 0.0224 - mean_squared_error: 0.0224
Epoch 2/3
6100/6100 [==============================] - 3s 573us/sample - loss: 0.0221 - mean_squared_error: 0.0221
Epoch 3/3
6100/6100 [==============================] - 3s 564us/sample - loss: 0.0223 - mean_squared_error: 0.0223


In [242]:
model.save("100_hidden_forced_distance_most_linear.h5")